## Import Libraries

In [1]:
import os
import webbrowser
from neo4j import GraphDatabase
from pyvis.network import Network
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time

## Load Data

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/movie-config.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Cypher Code

In [5]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
cypher = """
MATCH (p:Person {name: $name})-[r:ACTED_IN]->(m:Movie)
RETURN m.title AS title, r.role AS role
"""
name = "Tom Hanks"

records, summary, keys = driver.execute_query( # (1)
    cypher,    # (2)
    name=name  # (3)
)

In [7]:
print(keys)

['title', 'role']


In [8]:
print("Query Summary:")
print("Query Type:", summary.query_type)
print("Query Text:", summary.query)

print("\nCounters:")
for key, value in summary.counters.__dict__.items():
    if value:  # Only show non-zero counters
        print(f"{key}: {value}")

print("\nQuery Plan:")
if summary.plan:
    print(summary.plan)  # or use summary.plan.operator_type for plan root operator
else:
    print("No plan available.")

print("\nProfiled Plan:")
if summary.profile:
    print(summary.profile)  # or access attributes like profile.operator_type
else:
    print("No profile available.")

Query Summary:
Query Type: r
Query Text: 
MATCH (p:Person {name: $name})-[r:ACTED_IN]->(m:Movie)
RETURN m.title AS title, r.role AS role


Counters:

Query Plan:
No plan available.

Profiled Plan:
No profile available.


In [9]:
# RETURN m.title AS title, r.role AS role
for record in records:
    print("*" * 40)
    print(f"Title: {record['title']}")
    print(f"Role: {record['role']}")
    print("*" * 40)

****************************************
Title: Punchline
Role: Steven Gold
****************************************
****************************************
Title: Catch Me If You Can
Role: Carl Hanratty
****************************************
****************************************
Title: Dragnet
Role: Pep Streebeck
****************************************
****************************************
Title: Saving Mr. Banks
Role: Walt Disney
****************************************
****************************************
Title: Bachelor Party
Role: Rick Gassko
****************************************
****************************************
Title: Volunteers
Role: Lawrence Whatley Bourne III
****************************************
****************************************
Title: Man with One Red Shoe, The
Role: Richard Harlan Drew
****************************************
****************************************
Title: Splash
Role: Allen Bauer
****************************************
*

In [10]:
cypher = """
MATCH (m:Movie {title: $title})<-[:ACTED_IN]-(p)
RETURN p.name AS actor
"""
params = {"title": "Toy Story"}

res, summary, keys = driver.execute_query(

    cypher,
    **params
)

In [11]:
for record in res:
    print(record["actor"])

Jim Varney
Tim Allen
Tom Hanks
Don Rickles
